In [23]:
import numpy as np
import trimesh

# read unstructured mesh
uns_mesh: trimesh.Trimesh = trimesh.load_mesh('../../static/unsmesh/simple_tooth.obj')
str_mesh = trimesh.Trimesh()

uns_mesh.fill_holes()
uns_mesh.remove_duplicate_faces()

def vert_dist(msh: trimesh.Trimesh, vidx1: int, vidx2: int) -> float:
    return np.linalg.norm(
        msh.vertices[vidx1] - msh.vertices[vidx2]
    )

In [24]:
# get mesh inner and boundary vertices
# output:
#   - inn_verts
#   - bnd_verts
#   - bnd_length
import numpy_indexed as npi

bnd_edges = npi.difference(uns_mesh.edges_unique, uns_mesh.face_adjacency_edges)

bnd_verts = np.array([*bnd_edges[0]])
bnd_edges = np.delete(bnd_edges, [0], axis=0)
bnd_length = vert_dist(uns_mesh, *bnd_verts[:2])

success = True
while success:
    success = False
    last = bnd_verts[-1]
    for idx, edge in enumerate(bnd_edges):
        if last == edge[0]:
            success = True
            last = edge[1]
        elif last == edge[1]:
            success = True
            last = edge[0]
        if success:
            bnd_verts = np.append(bnd_verts, last)
            bnd_edges = np.delete(bnd_edges, [idx], axis=0)
            bnd_length += vert_dist(uns_mesh, *bnd_verts[-2:])
            break
# bnd_verts = np.unique(bnd_verts)

inn_verts = npi.difference(uns_mesh.face_adjacency_edges.flatten(), bnd_verts)

In [25]:
# parameterize bound to Square
# assume Z=0.0 in str_mesh
# output:
#   - f_B

from functools import reduce

_scale = 2 # square edge length

last_v = bnd_verts[0]
accumed = 0.

bnd_verts = bnd_verts[1:]
f_B = []

for bnd_v in bnd_verts:
    old_ratio = accumed / bnd_length
    accumed += vert_dist(uns_mesh, last_v, bnd_v)
    ratio = accumed / bnd_length
    flag = -reduce(
        lambda x, y: x * (1 if ((y - old_ratio) * (y - ratio)) > 0 else -y),
        [0.25, 0.5, 0.75],
        1
    )
    ratio = max(ratio, flag)
    vpos = (0., 0.)
    if ratio < 0.25:
        vpos = (-(_scale / 2) + _scale * (ratio / 0.25), -_scale / 2)
    elif ratio < 0.5:
        vpos = (_scale / 2,  -(_scale / 2) + _scale * ((ratio - 0.25) / 0.25))
    elif ratio < 0.75:
        vpos = ((_scale / 2) - _scale * ((ratio - 0.5) / 0.25), _scale / 2)
    else:
        vpos = (-_scale / 2, (_scale / 2) - _scale * ((ratio - 0.75) / 0.25))

    f_B.append(np.append(vpos, 0.))
    last_v = bnd_v

In [26]:
# initial weights
# keep row, col, data
from scipy.sparse import csc_matrix

def vectors_angle(msh: trimesh.Trimesh, mid: int, start: int, end: int) -> float:
    vec1: np.array = msh.vertices[start] - msh.vertices[mid]
    vec2: np.array = msh.vertices[end] - msh.vertices[mid]
    return np.arccos(vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))

def cot(angle: float) -> float:
    return np.cos(angle) / np.sin(angle)

sp_row = []
sp_col = []
sp_data = []

diag = np.zeros(len(uns_mesh.vertices))

for edge in uns_mesh.face_adjacency_edges:
    adj_list_s = uns_mesh.vertex_neighbors[edge[0]]
    adj_list_b = uns_mesh.vertex_neighbors[edge[1]]
    adj_vts = npi.intersection(adj_list_s, adj_list_b)
    if len(adj_vts) != 2:
        adj_vts = adj_vts[:2]
    # assert len(adj_vts) == 2, 'not a manifold'
    # compute cotangent weight of edge
    ang1 = vectors_angle(uns_mesh, adj_vts[0], *edge)
    ang2 = vectors_angle(uns_mesh, adj_vts[1], *edge)
    _weight = (cot(ang1) + cot(ang2)) / 2

    sp_row.append(edge[0])
    sp_col.append(edge[1])
    sp_data.append(-_weight)
    sp_row.append(edge[1])
    sp_col.append(edge[0])
    sp_data.append(-_weight)

    diag[edge[0]] += _weight
    diag[edge[1]] += _weight

# connect inn_verts and bnd_verts (ndarray)
tot_verts = np.append(inn_verts, bnd_verts)

print(len(tot_verts), len(uns_mesh.vertices))

sp_diag_index = tot_verts
sp_row.extend(sp_diag_index)
sp_col.extend(sp_diag_index)
sp_diag_data = [diag[v] for v in tot_verts] 

sp_data.extend(sp_diag_data)

sp_weights = csc_matrix((sp_data, (sp_row, sp_col)), dtype=float)

1181 1181


In [27]:
# solve linear system
# split L_{I,I} and L_{I,B}
len_inn = len(inn_verts)
len_bnd = len(bnd_verts)
sp_mid = sp_weights[inn_verts, ...]
sp_weights_II = sp_mid[..., inn_verts]
sp_weights_IB = sp_mid[..., bnd_verts]

In [28]:
from scipy.sparse.linalg import spsolve
# compute b = L_{BB}*f_B
assert sp_weights_IB.shape[1] == len(f_B), 'L_IB * f_B illegal'

b = -sp_weights_IB * f_B

# solve L_II * f_I = b
f_I = spsolve(sp_weights_II, b)

In [29]:
# rebuild str_mesh
# mapping: old to new index
import copy

param_bnd_verts = [v + len_inn for v in range(len_bnd)]
inv_mapping = dict(zip(bnd_verts, param_bnd_verts))
param_inn_verts = [v for v in range(len_inn)]
inv_mapping.update(zip(inn_verts, param_inn_verts))
# str_mesh.vertices = np.vstack([str_mesh.vertices, f_I])
param_tot = np.append(f_I, f_B, axis=0)
str_mesh.faces = copy.deepcopy(uns_mesh.faces)
str_mesh.vertices = [
    param_tot[inv_mapping[i]] for i in range(len_inn + len_bnd)
]

In [30]:
with open('save.obj', 'w') as ofile:
    str_mesh.export(ofile, 'obj')